# Map of Renewable Assets in the UK

The map idea is based on: http://ukdataexplorer.com/renewables/

There is also a plan to include production data using ROC info, ELEXON (BMRS) data, or data from ENTSOE. For example as done here: http://www.renewables-map.co.uk/

Map data is from: https://www.gov.uk/government/publications/renewable-energy-planning-database-monthly-extract
ROC data would be from: https://www.renewablesandchp.ofgem.gov.uk/Public/ReportManager.aspx?ReportVisibility=1&ReportCategory=0 

In [1]:
# import libraries

import pandas as pd # for dataframes
import numpy as np  # for arrays

from pyproj import Transformer # for coordinate transformation

from bokeh.plotting import figure, output_notebook, show, output_file # for plotting map
from bokeh.models import HoverTool
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.palettes import Category20, inferno, plasma, viridis
from bokeh.models import WMTSTileSource
from bokeh.models import CategoricalColorMapper, Legend

output_notebook() # to plot to notebook

Loading BokehJS ...

In [2]:
# import and clean data

# data from: https://www.gov.uk/government/publications/renewable-energy-planning-database-monthly-extract
file_name = r'renewable-energy-planning-database-q1-march-2021.csv'
renewablesData = pd.read_csv(file_name)

# only keep sites which are operational
renewablesData = renewablesData[pd.notnull(renewablesData['Operational'])]

# shorten the data set to useful parameters
coordinates = renewablesData[['Site Name','X-coordinate','Y-coordinate','Installed Capacity (MWelec)','Technology Type','Operational']]
coordinates = coordinates[pd.notnull(coordinates['X-coordinate'])]

In [3]:
# convert OSBG coordinates (Ordance Surver British National Grid)

# epsg coordinate system codes for transformations https://epsg.io/27700
#epsg:27700 is for OSBG
#epsg:3857 is for OpenMaps
#epsg:4326 is for longitude and latitude

TRANSFORM_27700_TO_3857 = Transformer.from_crs("EPSG:27700", "EPSG:3857")
TRANSFORM_27700_TO_4326 = Transformer.from_crs("EPSG:27700", "EPSG:4326")

coordinates['X-coordinateOM'],coordinates['Y-coordinateOM'] = TRANSFORM_27700_TO_3857.transform(coordinates['X-coordinate'].values,coordinates['Y-coordinate'].values)  

coordinates['Longitude'],coordinates['Latitude'] = TRANSFORM_27700_TO_4326.transform(coordinates['X-coordinate'].values,coordinates['Y-coordinate'].values)  

In [4]:
# create map plot

# See https://wiki.openstreetmap.org/wiki/Tile_servers for various tile services
MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}

tile_map = 'ESRI'

plot_width  = int(900)
plot_height = int(plot_width//1.2)
tools='pan,zoom_in,zoom_out,wheel_zoom,box_zoom,reset,save'

p = figure(tools=tools, 
        plot_width=plot_width, 
        plot_height=plot_height,
        sizing_mode="scale_both",
        x_axis_label="latitude",
        y_axis_label="longitude",
        match_aspect=True,
        x_axis_type="mercator",
        y_axis_type="mercator")


p.add_tile(MAP_TILES[tile_map])

p.add_layout(Legend(), 'right') # neat trick to make the legend appear outside the plot (https://discourse.bokeh.org/t/adding-a-legend-outside-the-plot-area-is-possible-even-with-auto-grouped-indirectly-created-legends/5595)

technologyTypes = coordinates['Technology Type'].unique()

colours = viridis(len(technologyTypes))

for tech,colour in zip(technologyTypes,colours):
    
    techData = coordinates.loc[coordinates['Technology Type'] == tech]
    
    source = ColumnDataSource(data=dict(
        x=techData['X-coordinateOM'],
        y=techData['Y-coordinateOM'],
        radius=2*techData['Installed Capacity (MWelec)'].values**0.5,
        sizeMW=techData['Installed Capacity (MWelec)'].values,
        technology=techData['Technology Type'],
        longitude=techData['Longitude'].values,
        latitude=techData['Latitude'].values,
        operational=techData['Operational'],
        siteName=techData['Site Name'],
    ))
    
    p.scatter('x', 'y',
          size='radius', 
          alpha=0.9, 
          fill_color=colour, 
          hover_fill_color=colour, 
          line_color="black",
          hover_line_color="white",
          fill_alpha=0.3, 
          hover_alpha=0.5,
          line_width=1, 
          legend_label=tech,
          source=source)

p.legend.click_policy="hide"

# add custom hover tool
hover = HoverTool(tooltips=[
    ("Site name","@siteName"),
    ("Technology", "@technology"),
    ("Size MW", "@sizeMW"),
    ("Operational","@operational")
    ])

p.add_tools(hover)
  

In [5]:
# show the plot
output_file('UK-Renewables.html')
show(p)

In [6]:
p = figure(tools=tools, 
        plot_width=plot_width, 
        plot_height=plot_height,
        sizing_mode="scale_width",
        x_axis_label="latitude",
        y_axis_label="longitude",
        match_aspect=True,
        x_axis_type="mercator",
        y_axis_type="mercator")


p.add_tile(MAP_TILES[tile_map])

p.add_layout(Legend(), 'right') # neat trick to make the legend appear outside the plot (https://discourse.bokeh.org/t/adding-a-legend-outside-the-plot-area-is-possible-even-with-auto-grouped-indirectly-created-legends/5595)

technologyTypes = ['Solar Photovoltaics', 'Wind Offshore', 'Wind Onshore']

colours = ['yellow','blue','green']

for tech,colour in zip(technologyTypes,colours):
    
    techData = coordinates.loc[coordinates['Technology Type'] == tech]
    
    source = ColumnDataSource(data=dict(
        x=techData['X-coordinateOM'],
        y=techData['Y-coordinateOM'],
        radius=2*techData['Installed Capacity (MWelec)'].values**0.5,
        sizeMW=techData['Installed Capacity (MWelec)'].values,
        technology=techData['Technology Type'],
        longitude=techData['Longitude'].values,
        latitude=techData['Latitude'].values,
        operational=techData['Operational'],
        siteName=techData['Site Name'],
    ))
    
    p.scatter('x', 'y',
          size='radius', 
          alpha=0.9, 
          fill_color=colour, 
          hover_fill_color=colour, 
          line_color="black",
          hover_line_color="white",
          fill_alpha=0.3, 
          hover_alpha=0.5,
          line_width=1, 
          legend_label=tech,
          source=source)

p.legend.click_policy="hide"

# add custom hover tool
hover = HoverTool(tooltips=[
    ("Site name","@siteName"),
    ("Technology", "@technology"),
    ("Size MW", "@sizeMW"),
    ("Operational","@operational")
    ])

p.add_tools(hover)

In [7]:
# show the plot
output_file('UK-WindAndSolar.html')
show(p) 